In [1]:
import pandas as pd
import os
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.preprocessing import MinMaxScaler, OneHotEncoder
import matplotlib.pyplot as plt
from sklearn.svm import SVC
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier, GradientBoostingClassifier
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.pipeline import Pipeline
from sklearn.model_selection import StratifiedKFold
from skopt import BayesSearchCV
from sklearn.metrics import r2_score, mean_absolute_error
from skopt.callbacks import DeltaYStopper
from skopt.space import Real, Categorical, Integer
import json
from xgboost import XGBClassifier
import keras
from keras.models import Sequential
from keras.layers import Dense
from keras.layers import Flatten
from keras.layers import Dropout
from keras.layers import LSTM
from keras.utils import to_categorical

2023-09-28 15:51:19.885720: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [2]:
import tensorflow as tf

In [3]:
# fit and evaluate a model
def evaluate_model(X_train, y_train, X_test, y_test):
    keras.backend.clear_session()
    verbose, epochs, batch_size = 1, 15, 64
    n_timesteps, n_features = X_train.shape[1], X_train.shape[2]
    model = Sequential()
    model.add(LSTM(100, input_shape=(n_timesteps,n_features),return_sequences=True))
    model.add(Dropout(0.5))
    model.add(Dense(100, activation='relu'))
    model.add(Dense(1, activation='sigmoid'))
    model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])
    # fit network
    model.fit(X_train, y_train, epochs=epochs, batch_size=batch_size, verbose=verbose)
    # evaluate model
    _, accuracy = model.evaluate(X_test, y_test, batch_size=batch_size, verbose=0)
    return accuracy
 
# summarize scores
def summarize_results(scores):
    print(scores)
    m, s = mean(scores), std(scores)
    print('Accuracy: %.3f%% (+/-%.3f)' % (m, s))
 
# run an experiment
def run_experiment(X_train, y_train, X_test, y_test, repeats=10):
    # load data
    #trainX, trainy, testX, testy = load_dataset()
    # repeat experiment
    scores = list()
    for r in range(repeats):
        score = evaluate_model(X_train, y_train, X_test, y_test)
        score = score * 100.0
        print('>#%d: %.3f' % (r+1, score))
        scores.append(score)
    # summarize results
    summarize_results(scores)

def load_data(input_df):
    
    array_list = list()
    target_list = list()
    for indx, row in input_df.iterrows():
        x0 = np.array(row['value_pad'])
        x1 = np.array(row['timedelta_pad'])
        array_list.append(np.column_stack([x0,x1]))
        target_list.append(row['readmit_flag'])
    
    #enc = OneHotEncoder()
    #y = enc.fit_transform(np.array(target_list).reshape(-1,1))
    
    #y = to_categorical(np.array(target_list), 2)
    
    return (np.array(array_list), np.array(target_list).reshape(-1,1))

In [4]:
RootDir = '/Users/ggray15/src/python/repos/jhach_cpdsam_symposium_workshop'
os.chdir(RootDir)

In [5]:
%run "./notebooks/function_library.ipynb"

In [6]:
input_df = pd.read_csv('data/chartevents.csv')
input_df

/var/folders/zt/m0rw2wgj3jb7012rstqhn7v80000gq/T/ipykernel_43311/3355962086.py:1: DtypeWarning: Columns (4,5,6,13,18,20,47) have mixed types. Specify dtype option on import or set low_memory=False.
  input_df = pd.read_csv('data/chartevents.csv')


,row_id,subject_id,gender,dob,dod,dod_hosp,dod_ssn,expire_flag,row_id.1,subject_id.1,...,charttime,storetime,cgid,value,valuenum,valueuom,warning,error,resultstatus,stopped
0,3335,3529,M,2199-04-20 00:00:00.000,NaN,NaN,NaN,0,4254,3529,...,2199-04-20 01:30:00.000,2199-04-20 01:34:00.000,17581,158.0,158.0,bpm,NaN,NaN,NaN,NotStopd
1,13569,14349,M,2157-12-27 00:00:00.000,NaN,NaN,NaN,0,17581,14349,...,2157-12-30 00:00:00.000,2157-12-29 23:58:00.000,20262,135.0,135.0,bpm,NaN,NaN,NaN,NotStopd
2,13569,14349,M,2157-12-27 00:00:00.000,NaN,NaN,NaN,0,17581,14349,...,2157-12-29 23:00:00.000,2157-12-29 22:20:00.000,20262,135.0,135.0,bpm,NaN,NaN,NaN,NotStopd
3,13569,14349,M,2157-12-27 00:00:00.000,NaN,NaN,NaN,0,17581,14349,...,2157-12-29 22:00:00.000,2157-12-29 22:20:00.000,20262,124.0,124.0,bpm,NaN,NaN,NaN,NotStopd
4,13569,14349,M,2157-12-27 00:00:00.000,NaN,NaN,NaN,0,17581,14349,...,2157-12-28 09:00:00.000,2157-12-28 08:45:00.000,16433,127.0,127.0,bpm,NaN,NaN,NaN,NotStopd
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1702894,23258,24648,F,2200-11-02 00:00:00.000,NaN,NaN,NaN,0,30099,24648,...,2200-11-02 15:00:00.000,2200-11-02 15:36:00.000,17125,146,146.0,bpm,NaN,NaN,NaN,NotStopd
1702895,23258,24648,F,2200-11-02 00:00:00.000,NaN,NaN,NaN,0,30099,24648,...,2200-11-02 14:00:00.000,2200-11-02 14:00:00.000,17125,130,130.0,bpm,NaN,NaN,NaN,NotStopd
1702896,23258,24648,F,2200-11-02 00:00:00.000,NaN,NaN,NaN,0,30099,24648,...,2200-11-02 13:00:00.000,2200-11-02 13:29:00.000,17125,122,122.0,bpm,NaN,NaN,NaN,NotStopd
1702897,23258,24648,F,2200-11-02 00:00:00.000,NaN,NaN,NaN,0,30099,24648,...,2200-11-02 12:30:00.000,2200-11-02 12:31:00.000,18246,117,117.0,bpm,NaN,NaN,NaN,NotStopd


In [7]:
time_df = input_df[['subject_id','hadm_id','itemid','charttime','value','valueuom']].sort_values(['hadm_id','charttime'])
time_df = time_df[time_df['itemid']==211]
time_df['charttime'] = time_df['charttime'].astype('datetime64[ns]')
time_df

,subject_id,hadm_id,itemid,charttime,value,valueuom
243116,1189,100023,211,2130-05-28 06:45:00,132.0,bpm
243115,1189,100023,211,2130-05-28 08:30:00,114.0,bpm
1162953,4528,100025,211,2191-07-13 12:00:00,144.0,bpm
1162926,4528,100025,211,2191-07-13 13:00:00,124.0,bpm
1162925,4528,100025,211,2191-07-13 14:00:00,126.0,bpm
...,...,...,...,...,...,...
613317,25768,199918,211,2111-05-28 02:30:00,154.0,bpm
613374,25768,199918,211,2111-05-28 07:00:00,148.0,bpm
613373,25768,199918,211,2111-05-28 10:30:00,156.0,bpm
901914,13446,199954,211,2120-11-08 12:00:00,142.0,bpm


In [8]:
count_df = time_df.groupby('hadm_id',as_index=False).agg({'value':'count'})
adm_list = count_df[count_df['value']>4]['hadm_id'].tolist()

In [9]:
time_df = time_df[time_df['hadm_id'].isin(adm_list)].reset_index()

In [10]:
def f_diff(t0, t1):
    return t1 - t0

t_df = time_df.groupby('hadm_id').apply(lambda x: f_diff(x.charttime.shift(), x.charttime)).reset_index(drop=True)

In [11]:
time_df['timedelta'] = t_df.tolist()
time_df['timedelta'] = time_df['timedelta'].fillna(pd.Timedelta(seconds=0))
time_df

,index,subject_id,hadm_id,itemid,charttime,value,valueuom,timedelta
0,1162953,4528,100025,211,2191-07-13 12:00:00,144.0,bpm,0 days 00:00:00
1,1162926,4528,100025,211,2191-07-13 13:00:00,124.0,bpm,0 days 01:00:00
2,1162925,4528,100025,211,2191-07-13 14:00:00,126.0,bpm,0 days 01:00:00
3,1162924,4528,100025,211,2191-07-13 15:00:00,119.0,bpm,0 days 01:00:00
4,1162937,4528,100025,211,2191-07-13 16:00:00,125.0,bpm,0 days 01:00:00
...,...,...,...,...,...,...,...,...
1692982,613277,25768,199918,211,2111-05-27 19:30:00,158.0,bpm,0 days 00:30:00
1692983,613318,25768,199918,211,2111-05-27 22:30:00,164.0,bpm,0 days 03:00:00
1692984,613317,25768,199918,211,2111-05-28 02:30:00,154.0,bpm,0 days 04:00:00
1692985,613374,25768,199918,211,2111-05-28 07:00:00,148.0,bpm,0 days 04:30:00


In [12]:
time_df['timedelta'] = time_df['timedelta'].astype('int') / 3600000000000 # convert to hours

In [13]:
mx = MinMaxScaler()
time_df['value'] = mx.fit_transform(time_df['value'].values.reshape(-1,1))

mx = MinMaxScaler()
time_df['timedelta'] = mx.fit_transform(time_df['timedelta'].values.reshape(-1,1))

time_df

,index,subject_id,hadm_id,itemid,charttime,value,valueuom,timedelta
0,1162953,4528,100025,211,2191-07-13 12:00:00,0.480000,bpm,0.000000
1,1162926,4528,100025,211,2191-07-13 13:00:00,0.413333,bpm,0.000922
2,1162925,4528,100025,211,2191-07-13 14:00:00,0.420000,bpm,0.000922
3,1162924,4528,100025,211,2191-07-13 15:00:00,0.396667,bpm,0.000922
4,1162937,4528,100025,211,2191-07-13 16:00:00,0.416667,bpm,0.000922
...,...,...,...,...,...,...,...,...
1692982,613277,25768,199918,211,2111-05-27 19:30:00,0.526667,bpm,0.000461
1692983,613318,25768,199918,211,2111-05-27 22:30:00,0.546667,bpm,0.002765
1692984,613317,25768,199918,211,2111-05-28 02:30:00,0.513333,bpm,0.003687
1692985,613374,25768,199918,211,2111-05-28 07:00:00,0.493333,bpm,0.004147


In [14]:
time_df.groupby('hadm_id',as_index=False).agg({'value':'count'}).median()

hadm_id    148982.0
value         165.0
dtype: float64

In [15]:
def pad_array(x0, cutoff=165):
    x_new = np.zeros(cutoff)
    n = np.min([cutoff, len(x0)])
    i = cutoff - n
    ii = len(x0) - n
    x_new[i:] = x0[ii:]
    return x_new

In [16]:
group_df = time_df.groupby('hadm_id', as_index=False).agg({'value':list, 'timedelta':list})
group_df

,hadm_id,value,timedelta
0,100025,"[0.48000000000000004, 0.4133333333333334, 0.42...","[0.0, 0.0009216589861751152, 0.000921658986175..."
1,100029,"[0.5133333333333334, 0.4666666666666667, 0.493...","[0.0, 0.0009216589861751152, 0.000921658986175..."
2,100044,"[0.5133333333333334, 0.4066666666666667, 0.5, ...","[0.0, 0.0007219662058371736, 0.000921658986175..."
3,100055,"[0.4666666666666667, 0.4666666666666667, 0.53,...","[0.0, 0.0, 0.0004608294930875576, 0.0, 0.00046..."
4,100062,"[0.52, 0.5133333333333334, 0.5133333333333334,...","[0.0, 0.0006912442396313364, 0.000230414746543..."
...,...,...,...
4549,199901,"[0.48333333333333334, 0.47000000000000003, 0.4...","[0.0, 0.0009216589861751152, 0.000921658986175..."
4550,199904,"[0.46, 0.4866666666666667, 0.5133333333333334,...","[0.0, 0.0002304147465437788, 0.000460829493087..."
4551,199913,"[0.5566666666666668, 0.5, 0.55, 0.516666666666...","[0.0, 0.0006912442396313364, 0.000921658986175..."
4552,199917,"[0.5933333333333334, 0.48000000000000004, 0.48...","[0.0, 0.0009216589861751152, 0.000921658986175..."


In [17]:
group_df['value_pad'] = group_df.value.apply(lambda x: pad_array(x, cutoff=165))
group_df['timedelta_pad'] = group_df.timedelta.apply(lambda x: pad_array(x, cutoff=165))

In [18]:
group_df.reset_index(drop=True,inplace=True)
group_df

,hadm_id,value,timedelta,value_pad,timedelta_pad
0,100025,"[0.48000000000000004, 0.4133333333333334, 0.42...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
1,100029,"[0.5133333333333334, 0.4666666666666667, 0.493...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.4466666666666667, 0.4666666666666667, 0.426...","[0.0009216589861751152, 0.0009216589861751152,..."
2,100044,"[0.5133333333333334, 0.4066666666666667, 0.5, ...","[0.0, 0.0007219662058371736, 0.000921658986175...","[0.48000000000000004, 0.5233333333333333, 0.54...","[0.0009216589861751152, 0.0009216589861751152,..."
3,100055,"[0.4666666666666667, 0.4666666666666667, 0.53,...","[0.0, 0.0, 0.0004608294930875576, 0.0, 0.00046...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ..."
4,100062,"[0.52, 0.5133333333333334, 0.5133333333333334,...","[0.0, 0.0006912442396313364, 0.000230414746543...","[0.5, 0.5166666666666667, 0.5066666666666667, ...","[0.0002304147465437788, 0.0002304147465437788,..."
...,...,...,...,...,...
4549,199901,"[0.48333333333333334, 0.47000000000000003, 0.4...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.5266666666666667, 0.49333333333333335, 0.5,...","[0.0009216589861751152, 0.0009216589861751152,..."
4550,199904,"[0.46, 0.4866666666666667, 0.5133333333333334,...","[0.0, 0.0002304147465437788, 0.000460829493087...","[0.49333333333333335, 0.5166666666666667, 0.51...","[0.0009216589861751152, 0.0009216589861751152,..."
4551,199913,"[0.5566666666666668, 0.5, 0.55, 0.516666666666...","[0.0, 0.0006912442396313364, 0.000921658986175...","[0.42333333333333334, 0.4366666666666667, 0.45...","[0.0004608294930875576, 0.0009216589861751152,..."
4552,199917,"[0.5933333333333334, 0.48000000000000004, 0.48...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.41000000000000003, 0.4766666666666667, 0.39...","[0.0009216589861751152, 0.0009216589861751152,..."


In [19]:
label_df = pd.read_csv('data/labels.csv')
label_df 

,hadm_id,readmit_flag
0,163353,0
1,178980,0
2,118037,1
3,118037,0
4,159514,0
...,...,...
8095,116323,0
8096,158613,0
8097,128876,0
8098,105824,0


In [20]:
merge_df = pd.merge(group_df, label_df, on='hadm_id', how='inner')
merge_df

,hadm_id,value,timedelta,value_pad,timedelta_pad,readmit_flag
0,100025,"[0.48000000000000004, 0.4133333333333334, 0.42...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
1,100029,"[0.5133333333333334, 0.4666666666666667, 0.493...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.4466666666666667, 0.4666666666666667, 0.426...","[0.0009216589861751152, 0.0009216589861751152,...",0
2,100044,"[0.5133333333333334, 0.4066666666666667, 0.5, ...","[0.0, 0.0007219662058371736, 0.000921658986175...","[0.48000000000000004, 0.5233333333333333, 0.54...","[0.0009216589861751152, 0.0009216589861751152,...",0
3,100055,"[0.4666666666666667, 0.4666666666666667, 0.53,...","[0.0, 0.0, 0.0004608294930875576, 0.0, 0.00046...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",1
4,100055,"[0.4666666666666667, 0.4666666666666667, 0.53,...","[0.0, 0.0, 0.0004608294930875576, 0.0, 0.00046...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...","[0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, 0.0, ...",0
...,...,...,...,...,...,...
4636,199901,"[0.48333333333333334, 0.47000000000000003, 0.4...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.5266666666666667, 0.49333333333333335, 0.5,...","[0.0009216589861751152, 0.0009216589861751152,...",0
4637,199904,"[0.46, 0.4866666666666667, 0.5133333333333334,...","[0.0, 0.0002304147465437788, 0.000460829493087...","[0.49333333333333335, 0.5166666666666667, 0.51...","[0.0009216589861751152, 0.0009216589861751152,...",0
4638,199913,"[0.5566666666666668, 0.5, 0.55, 0.516666666666...","[0.0, 0.0006912442396313364, 0.000921658986175...","[0.42333333333333334, 0.4366666666666667, 0.45...","[0.0004608294930875576, 0.0009216589861751152,...",0
4639,199917,"[0.5933333333333334, 0.48000000000000004, 0.48...","[0.0, 0.0009216589861751152, 0.000921658986175...","[0.41000000000000003, 0.4766666666666667, 0.39...","[0.0009216589861751152, 0.0009216589861751152,...",0


In [21]:
train_df, test_df = train_test_split(merge_df[['value_pad','timedelta_pad','readmit_flag']],test_size=0.4,random_state=42, stratify=merge_df['readmit_flag'].tolist())

In [22]:
X_train, y_train = load_data(train_df)
X_test, y_test = load_data(test_df)

In [23]:
X_train = np.nan_to_num(X_train, nan=0, posinf=1.1)

In [24]:
y_train.shape

(2784, 1)

In [25]:
run_experiment(X_train, y_train, X_test, y_test, repeats=10)

Epoch 1/15


2023-09-28 15:52:06.077294: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-28 15:52:06.079305: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-28 15:52:06.080786: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

44/44 [==============================] - 6s 85ms/step - loss: 0.2726 - accuracy: 0.9523
Epoch 2/15
44/44 [==============================] - 4s 88ms/step - loss: 0.1515 - accuracy: 0.9705
Epoch 3/15
44/44 [==============================] - 4s 86ms/step - loss: 0.1502 - accuracy: 0.9705
Epoch 4/15
44/44 [==============================] - 4s 85ms/step - loss: 0.1449 - accuracy: 0.9705
Epoch 5/15
44/44 [==============================] - 4s 85ms/step - loss: 0.1427 - accuracy: 0.9705
Epoch 6/15
44/44 [==============================] - 4s 90ms/step - loss: 0.1426 - accuracy: 0.9705
Epoch 7/15
44/44 [==============================] - 4s 87ms/step - loss: 0.1400 - accuracy: 0.9705
Epoch 8/15
44/44 [==============================] - 4s 85ms/step - loss: 0.1415 - accuracy: 0.9705
Epoch 9/15
44/44 [==============================] - 4s 85ms/step - loss: 0.1383 - accuracy: 0.9705
Epoch 10/15
44/44 [==============================] - 4s 86ms/step - loss: 0.1386 - accuracy: 0.9705
Epoch 11/15
44/44 [=

2023-09-28 15:53:06.246395: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-28 15:53:06.248454: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-28 15:53:06.250031: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

>#1: 97.092
Epoch 1/15


...

2023-09-28 15:55:49.716837: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-28 15:55:49.718866: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-28 15:55:49.720335: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

Epoch 1/15


2023-09-28 15:55:50.190769: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_2_grad/concat/split_2/split_dim' with dtype int32
	 [[{{node gradients/split_2_grad/concat/split_2/split_dim}}]]
2023-09-28 15:55:50.193247: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'gradients/split_grad/concat/split/split_dim' with dtype int32
	 [[{{node gradients/split_grad/concat/split/split_dim}}]]
2023-09-28 15:55:50.195133: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You mus

44/44 [==============================] - 12s 187ms/step - loss: 0.2774 - accuracy: 0.9627
Epoch 2/15
44/44 [==============================] - 5s 118ms/step - loss: 0.1496 - accuracy: 0.9705
Epoch 3/15
44/44 [==============================] - 6s 139ms/step - loss: 0.1469 - accuracy: 0.9705
Epoch 4/15
27/44 [=================>............] - ETA: 2s - loss: 0.1390 - accuracy: 0.9722

KeyboardInterrupt: 

In [ ]:
y_train.shape

In [ ]:
X_test.shape